# Reproduce SimPEG(emg3d) with pyGIMLi(emg3d)

Requires
- `simpeg>=1.5.2`
- `emg3d@inv-pygimli` (`pip install git+https://github.com/emsig/emg3d@inv-pygimli`

Super simple plus all possible inv-crimes (same mesh, some solver, ...).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import emg3d
import emg3d.inversion.pygimli as ipygimli

%run ../data/sampledata

In [ ]:
%matplotlib widget

## Load Simulation (incl. Survey, Models, Data)

In [ ]:
sim, model = load_data("../data/SimPEG-Original")
model_init = sim.model.copy()

# sim.tol_gradient = 1e-2  # could be reduced further to potentially speed-up more

## Define regions, instantiate Kernel and Inversion

In [ ]:
# Create region markers (as many different regions as you like)
markers = np.zeros(sim.model.shape, dtype=int)
markers[:, :, :-6] = 1

# Instantiate the emg3d-kernel for pyGIMLi
fop = emg3d.inversion.pygimli.Kernel(simulation=sim, markers=markers, pgthreads=1)

# Instantiate the inversion and set some inversion parameters
INV = emg3d.inversion.pygimli.Inversion(fop=fop)
#INV.inv.setCGLSTolerance(1e-3)  # is _absolute_, not _relative_
INV.inv.setMaxCGLSIter(30)
#INV.dataTrans = pg.trans.TransSymLog(sim.survey.noise_floor)

INV.fop.setRegionProperties(0, limits=(0.0001, 2), startModel=1)                  # Target
INV.fop.setRegionProperties(1, background=True)                                   # Background

#INV.stopAtChi1 = False

## Run inversion

In [ ]:
INV.run(
    maxIter=20, # just to test
    lam=10,  # btw 1-100
    #verbose=True,
    #debug=True,
    #startModel=model_true.property_x.ravel('F'),
    #isReference=True,
)

## Plots

In [ ]:
plot_models(sim, sim.model, model)

In [ ]:
plot_responses(sim)

In [ ]:
emg3d.Report()